In [1]:
import mxnet as mx
import numpy as np
from sklearn.cluster import KMeans
import time

/home/tapir/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def convolve_codebook_ndarray(data, codebook):
    filters = codebook
    fshape  = filters.shape
    
    filters = mx.nd.transpose(filters, axes=(1,0,2,3)).reshape((-1,1,0, 0)) 
    res = mx.nd.Convolution(data=data, weight=filters, num_group=fshape[1], num_filter=fshape[0]*fshape[1], no_bias=True, kernel=(3,3))
    res = res.expand_dims(1)
    res = res.reshape((0,fshape[1],fshape[0], 0, 0))
    res = mx.nd.transpose(res,axes=(0,2,1,3,4))
    
    return res

In [3]:
def quantize(data,shrink=8):
    shape=data.shape
    result=np.zeros(shape)
    nclusters=shape[0]/shrink
    q_indices=np.zeros((shape[0],shape[1]))
    cluster_centers=np.zeros((nclusters, shape[1], shape[2], shape[3]))
    
    for channel in range(shape[1]):
        c_data=data[:,channel,:,:]
        cshape=c_data.shape
        c_data_shaped=c_data.reshape((cshape[0], cshape[1]*cshape[2]))
        
        estimator = KMeans(n_clusters=nclusters)
        estimator.fit(c_data_shaped.asnumpy())
        
        indices = estimator.predict(X=c_data_shaped.asnumpy())
        data_quantized = np.array([estimator.cluster_centers_[idx] for idx in indices])

        cluster_centers[:,channel,:,:] = estimator.cluster_centers_.reshape(nclusters,cshape[1],cshape[2])
        q_indices[:,channel]=indices
        
        result[:,channel,:,:]=data_quantized.reshape(cshape)
    
    return result, cluster_centers, q_indices

In [4]:
def convolve_codebook(data, indices, codebookshape, output_shape):
    filters = mx.sym.Variable("codebook", shape=codebookshape)
    fshape  = codebookshape #4,16,3,3
    index_shape=indices.shape
    
    filters = mx.sym.transpose(filters, axes=(1,0,2,3)).reshape((-1,1,0, 0)) #TODO: transpose is unnecessary!!
    res = mx.sym.Convolution(data=data, weight=filters, num_group=fshape[1], num_filter=fshape[0]*fshape[1], no_bias=True, kernel=(3,3))
    res = res.expand_dims(1)
    res = res.reshape((0,fshape[1],fshape[0], 0, 0))
    res = mx.sym.transpose(res,axes=(0,2,1,3,4)) #lookup table
    
    #hacky because multi-dim indexing isn't allowed
    res = mx.sym.reshape(data=res,shape=(-1,0),reverse=1) #(sample*nclusters*channel*W,H)
    #now looking up the results
    
    #print res[0,1,0] #7, 4, 16 ,30, 30
    print index_shape#7,4,16,30,30
    lres=[]
    #TODO: find a way to implement with less loops
    for sample in range(output_shape[0]):
        filterwise_list=[]
        for fltr in range(index_shape[0]):
            channelwise_list=[]
            for ch in range(index_shape[1]):
                            ## (((sample*4+cluster)*channels)*channel)*width
                slice_begin = (((sample*fshape[0]+indices[fltr,ch])*fshape[1]+ch)*output_shape[2],0)
                slice_end   = (slice_begin[0]+output_shape[2],output_shape[3])

                #channelwise_list.append(res[sample][indices[fltr,ch]][ch][0])
                channelwise_list.append(mx.sym.slice(data=res, begin=slice_begin, end=slice_end))
                
            filterwise_list.append(mx.sym.sum(mx.sym.stack(*channelwise_list),axis=0))
        lres.append(mx.sym.stack(*filterwise_list))
    lres=mx.sym.stack(*lres)                 
                
    
    
    return lres

In [5]:
img=mx.nd.random.uniform(0, 1, shape=(8,64,32,32))
labels=mx.nd.array([1,0,1,0,0,1,0,1])
fshape=(128,64,3,3)
orig_filter=mx.nd.random.uniform(0, 1, shape=fshape)

qfilter, codebook_filter, indices = quantize(orig_filter)
indices=indices.astype(int)
codebookshape=codebook_filter.shape
indices_shape=indices.shape
data_iter = mx.io.NDArrayIter(img, batch_size= 4)


In [6]:
args={"codebook": mx.nd.array(codebook_filter)}
data=mx.sym.Variable("data")


In [ ]:
sym=convolve_codebook(data=data,indices= indices,codebookshape= codebookshape, output_shape=(4,128,30,30))
mod=mx.mod.Module(symbol=sym, context=mx.gpu())
#mod.init_params(arg_params=args)
mod.bind(for_training=False, data_shapes=data_iter.provide_data)#,{'codebook': mx.nd.array(codebook_filter)}], label_shapes=None)
mod.set_params(args,None)

In [ ]:
begin=time.time()

mx.profiler.set_config(profile_all=True,
                        filename='clustered_profile.json',  # File used for chrome://tracing visualization
                        continuous_dump=True,
                        aggregate_stats=True,
                      profile_symbolic=True)
mx.profiler.set_state('run')

result=mod.predict(eval_data=data_iter).asnumpy()

mx.profiler.set_state('stop')
print(mx.profiler.dumps())

#print time.time()-begin


In [ ]:
begin=time.time()
result=mod.predict(eval_data=data_iter).asnumpy()
print time.time()-begin

In [ ]:
data_iter.reset()

In [ ]:
##baseline
args2={"filters":mx.nd.array(qfilter)}

baseline_filters=mx.sym.Variable("filters")
sym2=mx.sym.Convolution(data=data,weight=baseline_filters, num_filter=128, kernel=(3,3), no_bias=True)

mod2=mx.mod.Module(symbol=sym2, context=mx.gpu())
#mod.init_params(arg_params=args)
mod2.bind(for_training=False, data_shapes=data_iter.provide_data)#,{'codebook': mx.nd.array(codebook_filter)}], label_shapes=None)
mod2.set_params(args2, None)

In [ ]:
begin=time.time()

mx.profiler.set_config(profile_all=True,
                        filename='baseline_profile.json',  # File used for chrome://tracing visualization
                        continuous_dump=True,
                        aggregate_stats=True)
mx.profiler.set_state('run')

result2=mod2.predict(eval_data=data_iter).asnumpy()

mx.profiler.set_state('stop')
print(mx.profiler.dumps())
print time.time()-begin



In [ ]:
np.mean(np.square(result-result2))

In [ ]:
data_iter.reset()

In [7]:
##baseline2
reshaped_filter = mx.nd.transpose(mx.nd.array(codebook_filter), axes=(1,0,2,3)).reshape((-1,1,0, 0))
args3={"filters":reshaped_filter}

baseline3_filters=mx.sym.Variable("filters")
sym3= mx.sym.Convolution(data=data, weight=baseline3_filters, num_group=codebookshape[1], 
                         num_filter=codebookshape[0]*codebookshape[1], no_bias=True, kernel=(3,3))


mod3=mx.mod.Module(symbol=sym3, context=mx.gpu())
#mod.init_params(arg_params=args)
mod3.bind(for_training=False, data_shapes=data_iter.provide_data)#,{'codebook': mx.nd.array(codebook_filter)}], label_shapes=None)
mod3.set_params(args3, None)

/home/tapir/miniconda2/lib/python2.7/site-packages/mxnet/module/base_module.py:54: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
	filters
  warnings.warn(msg)
/home/tapir/miniconda2/lib/python2.7/site-packages/mxnet/module/base_module.py:66: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [8]:

mx.profiler.set_config(profile_all=True,
                        filename='baseline_profile.json',  # File used for chrome://tracing visualization
                        continuous_dump=True,
                        aggregate_stats=True)
mx.profiler.set_state('run')
begin=time.time()

result3=mod3.predict(eval_data=data_iter).asnumpy()
print time.time()-begin

mx.profiler.set_state('stop')
print(mx.profiler.dumps())



0.0629680156708

Profile Statistics.
	Note that counter items are counter values and not time units.
Device Storage
Name                          Total Count        Time (ms)    Min Time (ms)    Max Time (ms)    Avg Time (ms)
----                          -----------        ---------    -------------    -------------    -------------
Memory: gpu/0                           8       14751.0117       14751.0117       58987.8125       22118.4004

MXNET_C_API
Name                          Total Count        Time (ms)    Min Time (ms)    Max Time (ms)    Avg Time (ms)
----                          -----------        ---------    -------------    -------------    -------------
MXNDArraySyncCopyToCPU                  1          60.1530          60.1530          60.1530          60.1530
MXNDArrayGetDType                       5           0.0030           0.0000           0.0010           0.0006
MXNet C API Calls                      70           0.0700           0.0010           0.0700         

In [ ]:
result.shape

In [ ]:
result2=convolve_codebook_ndarray(img,args["codebook"])
np.array_equal(result.asnumpy(),result2.asnumpy())

In [ ]:
import numpy as np

In [ ]:
aa=mx.nd.zeros((7,4,16,30,30))

for s in range(7):
    for i in range(4):
        filled= np.empty((16,30,30))
        filled.fill(i+s)
        aa[s,i,:,:,:] = mx.nd.array(filled)

In [ ]:
lookup_idx=mx.nd.array(np.random.choice(4,(32,16)))
print lookup_idx[0].shape

In [ ]:
lres=mx.nd.tile(data=mx.nd.zeros_like(aa[0,0,0,:,:]),reps=(7,32,1,1))
lres.shape

In [ ]:
aa[0][0][0].shape

In [ ]:
#lres=mx.nd.zeros((7,32,30,30))

for sample in range(7):#7
    for fltr in range(lookup_idx.shape[0]):#32 
        for ch in range(lookup_idx.shape[1]):#16
            lres[sample,fltr]+=aa[sample,lookup_idx[fltr,ch],ch,:,:][0]

In [ ]:
#this is how you slice ------------------------------------
print aa[1,3,15,0]
print aa.shape
samp=1
cls=3
ch=15
print ((samp*4+cls)*16+ch)*30
aa2=mx.nd.reshape(data=aa,shape=(-1,0),reverse=1)
print mx.nd.slice(data=aa2, begin=(((samp*4+cls)*16+ch)*30,0), end=(((samp*4+cls)*16+ch)*30+30,30))
#print aa2[480*7:480*7+30]
#print mx.nd.slice_axis(data=aa,axis=2,begin=0,end=1).shape

In [ ]:
lres2=[]
for sample in range(7):#7
    filterwise_list=[]
    for fltr in range(lookup_idx.shape[0]):#32 
        channelwise_list=[]
        for ch in range(lookup_idx.shape[1]):#16
            
            channelwise_list.append(aa[sample,lookup_idx[fltr,ch],ch,:,:][0])
        filterwise_list.append(mx.nd.sum(mx.nd.stack(*channelwise_list),axis=0))
    lres2.append(mx.nd.stack(*filterwise_list))
lres2=mx.nd.stack(*lres2)
print lres2.shape

In [ ]:
c=mx.nd.stack(c,b, axis = 1)
c.shape

In [ ]:
mx.nd.tile(data=a,reps=(7,16,1,1)).shape

In [ ]:
mx.nd.sum(lookup_idx[6])

In [ ]:
lres2[0][6][0]

In [ ]:
img=mx.nd.random.uniform(0, 1, shape=(8,16,32,32))
data_iter = mx.io.NDArrayIter(img, batch_size= 3, label_name=None)


In [ ]:
data_iter.provide_data

In [ ]:
data_iter.next()

In [ ]:
data_iter.provide_data

In [ ]:
def construct_tensors(data, l_indices, index_shape, batch_size):
    lookup_idx = mx.sym.Variable("indices", shape=indes_shape)
  
    lres=mx.nd.tile(data=mx.nd.zeros_like(data[0,0,0,:,:]),reps=(7,32,1,1))
    
    for sample in range(batch_size):
        for fltr in range(indes_shape[0]):
            for ch in range(index_shape[1]):
                lres[sample,fltr]+=aa[sample,lookup_idx[fltr,ch],ch,:,:][0]
    

In [ ]:
def convolve_codebook_nd(data, indices, codebook, codebookshape, output_shape):
    #filters = mx.sym.Variable("codebook", shape=codebookshape)
    filters=codebook
    
    fshape  = codebookshape #4,16,3,3
    index_shape=indices.shape
    
    filters = mx.nd.transpose(filters, axes=(1,0,2,3)).reshape((-1,1,0, 0)) #TODO: transpose is unnecessary!!
    res = mx.nd.Convolution(data=data, weight=filters, num_group=fshape[1], num_filter=fshape[0]*fshape[1], no_bias=True, kernel=(3,3))
    res = res.expand_dims(1)
    res = res.reshape((0,fshape[1],fshape[0], 0, 0))
    res = mx.nd.transpose(res,axes=(0,2,1,3,4)) #lookup table
    
    #hacky because multi-dim indexing isn't allowed
    res = mx.nd.reshape(data=res,shape=(-1,0),reverse=1) #(sample*nclusters*channel*W,H)
    #now looking up the results
    
    #print res[0,1,0] #7, 4, 16 ,30, 30
    #print index_shape#7,4,16,30,30
    lres=[]
    #TODO: find a way to implement with less loops
    for sample in range(output_shape[0]):
        filterwise_list=[]
        for fltr in range(index_shape[0]):
            channelwise_list=[]
            for ch in range(index_shape[1]):
                            ## (((sample*4+cluster)*channels)*channel)*width
                slice_begin = (((sample*fshape[0]+indices[fltr,ch])*fshape[1]+ch)*output_shape[2],0)
                slice_end   = (slice_begin[0]+output_shape[2],output_shape[3])
                
                #print slice_begin
                #print slice_end
                #channelwise_list.append(res[sample][indices[fltr,ch]][ch][0])
                channelwise_list.append(mx.nd.slice(data=res, begin=slice_begin, end=slice_end))
                
            filterwise_list.append(mx.nd.sum(mx.nd.stack(*channelwise_list),axis=0))
        lres.append(mx.nd.stack(*filterwise_list))
    lres=mx.nd.stack(*lres)                 
                
    
    
    return lres

In [ ]:
import time

In [ ]:
img=mx.nd.random.uniform(0, 1, shape=(10,64,32,32))
fshape=(128,64,3,3)
orig_filter=mx.nd.random.uniform(0, 1, shape=fshape)

qfilter, codebook_filter, indices = quantize(orig_filter)
indices=indices.astype(int)
codebook_filter=mx.nd.array(codebook_filter)
codebookshape=codebook_filter.shape
indices_shape=indices.shape

In [ ]:
begin=time.time()
print"======================="
well = convolve_codebook_nd(data=img,codebook=codebook_filter,indices=indices,output_shape=(10,128,30,30), codebookshape=codebookshape)
print  well.asnumpy().shape
print time.time()-begin

In [ ]:
well.shape

In [ ]:
begin=time.time()
print "========================="
well2=mx.nd.Convolution(data=img,weight=mx.nd.array(qfilter),kernel=(3,3),num_filter=128, no_bias=True)
print  well2.asnumpy().shape
print time.time()-begin

In [ ]:
well2.shape

In [ ]:
mx.nd.mean(well-well2)

In [ ]:
well2[0]

In [ ]:
img=mx.nd.random.uniform(0, 1, shape=(8,16,32,32))
labels=mx.nd.array([1,0,1,0,0,1,0,1])
labels = mx.nd.one_hot(depth=2,indices=labels)
fshape=(32,16,3,3)
orig_filter=mx.nd.random.uniform(0, 1, shape=fshape)

qfilter, codebook_filter, indices = quantize(orig_filter)
indices=indices.astype(int)
codebookshape=codebook_filter.shape
indices_shape=indices.shape
data_iter = mx.io.NDArrayIter(img, batch_size= 4, label=labels)




In [ ]:
args={"codebook": mx.nd.array(codebook_filter)}
data=mx.sym.Variable("data")
sym=convolve_codebook(data=data,indices= indices,codebookshape= codebookshape, output_shape=(4,32,30,30))
sym=mx.sym.Flatten(sym)
sym = mx.symbol.FullyConnected(data=sym, num_hidden=2, no_bias=True)
sym = mx.symbol.SoftmaxOutput(data=sym, name='softmax')

mod=mx.mod.Module(symbol=sym, context=mx.cpu())
#mod.init_params(arg_params=args)
mod.bind( data_shapes=data_iter.provide_data, label_shapes=data_iter.provide_label)#,{'codebook': mx.nd.array(codebook_filter)}], label_shapes=None)
mod.set_params(args,None,allow_missing=True)

In [ ]:
optimizer_params = {'learning_rate': 0.0000001,
                       'momentum': 0.9,
                       'wd': 0.0005,
                       'clip_gradient': None,
                       'rescale_grad': 1.0}

mod.fit(data_iter, eval_data=data_iter, arg_params=args,eval_metric=['acc'],optimizer='sgd',num_epoch=10
        ,optimizer_params=optimizer_params
               )

In [ ]:
mod.predict(data_iter)

In [ ]:
data_iter.reset()

In [ ]:
newargs=mod.get_params()

In [ ]:
newargs[0]['codebook'].shape